In [58]:
import pandas as pd
import os
import datetime
to_datetime = lambda d: pd.to_datetime(d, format='%d/%m/%Y')

In [28]:
fiis_names = []
fiis_dataFrame = pd.DataFrame()
dir = "../cotacoes/dados/b3/cotacoes_diarias/"
for file in os.listdir(dir):
    if file.endswith("11.sa.csv"):
        fiis_names.append(file[:6])
        fiis_dataFrame = fiis_dataFrame.merge(pd.read_csv(dir + file, converters={'date': to_datetime}).set_index('date').add_suffix('_'+file[:6]), how='outer', left_index=True, right_index=True)
            
fiis_dataFrame.tail()

,open_ALMI11,high_ALMI11,low_ALMI11,close_ALMI11,neg_ALMI11,vol_ALMI11,open_ALUP11,high_ALUP11,low_ALUP11,close_ALUP11,...,low_XPML11,close_XPML11,neg_XPML11,vol_XPML11,open_XTED11,high_XTED11,low_XTED11,close_XTED11,neg_XTED11,vol_XTED11
date,,,,,,,,,,,,,,,,,,,,,
2019-08-22,1475.00,1475.00,1452.02,1471.00,37.0,29886129.0,25.61,25.80,25.47,25.80,...,111.11,111.69,1328.0,361339304.0,6.62,6.69,6.60,6.62,65.0,1199087.0
2019-08-23,1493.99,1493.99,1460.00,1467.00,15.0,9245300.0,25.73,25.73,25.14,25.40,...,110.13,110.85,1967.0,506810955.0,6.70,7.05,6.69,6.90,136.0,1067034.0
2019-08-26,1460.11,1472.63,1450.00,1472.63,28.0,15773122.0,25.79,25.79,24.86,25.10,...,110.12,110.94,1744.0,370917899.0,6.98,7.30,6.50,6.85,275.0,9159779.0
2019-08-27,1450.20,1452.00,1430.00,1430.00,25.0,10983683.0,25.20,25.49,24.76,24.99,...,110.12,110.75,1424.0,454071646.0,6.86,7.00,6.81,6.81,87.0,1401120.0
2019-08-28,1429.99,1446.00,1420.02,1446.00,18.0,8719375.0,24.94,25.50,24.78,25.22,...,110.02,111.45,1090.0,356731930.0,6.89,6.90,6.56,6.60,121.0,3114215.0


In [30]:
start = to_datetime('28/08/2018')
fiis = fiis_dataFrame[fiis_dataFrame.index >= start]
fiis.dropna(axis=1, how='all')
#fiis.plot(figsize=(15,6))

,open_ALMI11,high_ALMI11,low_ALMI11,close_ALMI11,neg_ALMI11,vol_ALMI11,open_ALUP11,high_ALUP11,low_ALUP11,close_ALUP11,...,low_XPML11,close_XPML11,neg_XPML11,vol_XPML11,open_XTED11,high_XTED11,low_XTED11,close_XTED11,neg_XTED11,vol_XTED11
date,,,,,,,,,,,,,,,,,,,,,
2018-08-28,1203.01,1203.01,1150.00,1170.01,53.0,32793861.0,15.65,15.85,15.48,15.75,...,89.25,90.83,214.0,4.936458e+07,16.80,16.90,16.80,16.90,9.0,1957646.0
2018-08-29,1199.80,1200.00,1172.02,1200.00,179.0,86294315.0,15.68,15.85,15.50,15.57,...,89.51,90.26,286.0,3.160376e+07,16.90,16.90,16.68,16.70,12.0,1517970.0
2018-08-30,1224.99,1297.99,1212.01,1222.00,80.0,33467775.0,15.56,15.62,15.21,15.23,...,89.20,89.85,148.0,3.366955e+07,16.70,16.90,16.70,16.90,9.0,2559975.0
2018-08-31,1222.17,1300.00,1222.17,1265.25,213.0,54003283.0,15.26,15.41,15.15,15.20,...,89.65,90.50,112.0,3.381709e+07,16.74,17.99,16.57,17.99,24.0,2891217.0
2018-09-03,1266.01,1329.99,1266.01,1300.00,56.0,21529035.0,15.18,15.28,15.11,15.11,...,87.50,88.18,230.0,3.768062e+07,17.50,17.50,16.58,17.45,37.0,4262006.0
2018-09-04,1300.00,1309.99,1258.01,1283.04,47.0,17694895.0,15.10,15.27,15.00,15.07,...,87.97,88.42,126.0,2.718953e+07,16.72,17.39,16.72,17.39,30.0,2329005.0
2018-09-05,1290.00,1310.00,1258.00,1262.00,49.0,19462486.0,15.05,15.30,15.01,15.10,...,86.00,86.40,299.0,6.924171e+07,16.90,17.39,16.90,17.38,30.0,1538338.0
2018-09-06,1270.03,1309.99,1265.00,1300.00,25.0,7664797.0,15.22,15.26,15.09,15.24,...,85.03,86.18,162.0,4.250712e+07,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-10,1303.99,1303.99,1265.01,1275.02,18.0,3566695.0,15.29,15.51,15.28,15.50,...,85.80,86.54,190.0,5.353430e+07,17.01,17.01,16.93,16.93,20.0,1042654.0


In [53]:
fiis_closing_key = []

for key in fiis.keys():
    if(key.startswith('close')):
        fiis_closing_key.append(key)
        
fiis_closing_key

#'g' in df.index

['close_ALMI11',
 'close_ALUP11',
 'close_ALZR11',
 'close_AQLL11',
 'close_ATCR11',
 'close_ATSA11',
 'close_BARI11',
 'close_BBAS11',
 'close_BBIM11',
 'close_BBPO11',
 'close_BBRC11',
 'close_BBVJ11',
 'close_BCFF11',
 'close_BCIA11',
 'close_BCRI11',
 'close_BIDI11',
 'close_BMII11',
 'close_BNFS11',
 'close_BOVA11',
 'close_BPAC11',
 'close_BPFF11',
 'close_BPRP11',
 'close_BRCR11',
 'close_BRGE11',
 'close_BTCR11',
 'close_BVAR11',
 'close_BZLI11',
 'close_CARE11',
 'close_CBOP11',
 'close_CEOC11',
 'close_CNES11',
 'close_CRFF11',
 'close_CTXT11',
 'close_CXRI11',
 'close_CXTL11',
 'close_DMAC11',
 'close_DOMC11',
 'close_EDGA11',
 'close_ENGI11',
 'close_EURO11',
 'close_FAED11',
 'close_FCFL11',
 'close_FEXC11',
 'close_FFCI11',
 'close_FIGS11',
 'close_FIIB11',
 'close_FINF11',
 'close_FISC11',
 'close_FISD11',
 'close_FIVN11',
 'close_FLMA11',
 'close_FLRP11',
 'close_FMOF11',
 'close_FOFT11',
 'close_FPAB11',
 'close_FPNG11',
 'close_FVBI11',
 'close_FVPQ11',
 'close_GGRC11

In [135]:
fiis_per_month = 3

delta = datetime.timedelta(days=20)
skip = datetime.timedelta(days=1)

variation = []
fiis_to_buy = []

past = start
current = past + delta
while current < fiis.index[-1]:
    while not current in fiis.index and current < fiis.index[-1]:
        current = current + skip
    # here happens the magic
    
    variation = []
    current_data = fiis.loc[current]
    past_data = fiis.loc[past]
    
    for f in fiis_closing_key:
        variation.append((current_data[f] - past_data[f])/past_data[f])
        
    fiis_to_buy = []
    for ftb in range(fiis_per_month):
        index = variation.index(min(variation))
        variation[index] = 10000 # big value for elimination sake
        fiis_to_buy.append(fiis_closing_key[index])
        
     
    print(fiis_to_buy)
    past = current
    current += delta

TypeError: 'list' object cannot be interpreted as an integer

In [76]:
current = start
current = start + datetime.timedelta(days=400)
print(current)
print(fiis.index[-1])
print(current < fiis.index[-1])

2019-10-02 00:00:00
2019-08-28 00:00:00
False


In [492]:

fiis_dataFrameTest = pd.DataFrame()

test2 = pd.read_csv(dir + 'ALUP11.sa.csv', converters={'date': to_datetime}).set_index('date')
test2 = test2.add_prefix('oi')
print(test2.tail())

fiis_dataFrameTest = fiis_dataFrameTest.merge( test2, how='outer', left_index=True, right_index=True, suffixes=('', '2'))

print(fiis_dataFrameTest.tail())


            oiopen  oihigh  oilow  oiclose  oineg       oivol
date                                                         
2019-08-22   25.61   25.80  25.47    25.80   1868   787540300
2019-08-23   25.73   25.73  25.14    25.40   2088   955703900
2019-08-26   25.79   25.79  24.86    25.10   2317  1025724100
2019-08-27   25.20   25.49  24.76    24.99   2080  1261621400
2019-08-28   24.94   25.50  24.78    25.22   2518  1139040400
            oiopen  oihigh  oilow  oiclose  oineg       oivol
date                                                         
2019-08-22   25.61   25.80  25.47    25.80   1868   787540300
2019-08-23   25.73   25.73  25.14    25.40   2088   955703900
2019-08-26   25.79   25.79  24.86    25.10   2317  1025724100
2019-08-27   25.20   25.49  24.76    24.99   2080  1261621400
2019-08-28   24.94   25.50  24.78    25.22   2518  1139040400


In [134]:
test = [1, 2, -3, 4, 5]

test.index(min(test))

2